In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import models
import os
import numpy as np

In [ ]:
# Uncomment the line corresponding to the desired model (ResNet50 or ViT-B/16)

# neuralnet = models.resnet50(pretrained=True)
# neuralnet = models.vit_b_16(weights='IMAGENET1K_V1')

neuralnet = neuralnet.to('cuda')
neuralnet.eval() 

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256), 
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = torchvision.datasets.ImageFolder(root="add_test_set_path_here", transform=transform)

def torch_imshow(img):
    mean = torch.tensor([0.485, 0.456, 0.406], device=img.device)
    std = torch.tensor([0.229, 0.224, 0.225], device=img.device)
    img = img.permute(1, 2, 0)
    img = img * std + mean    
    img = torch.clamp(img, 0, 1)
    plt.imshow(img.cpu().numpy())
    plt.show()

cat_img, cat_label = test_dataset[1]
torch_imshow(cat_img)
print(f'Label for cat: {cat_label} - {test_dataset.classes[cat_label]}')

dog_img, dog_label = test_dataset[-1]
torch_imshow(dog_img)
print(f'Label for dog: {dog_label} - {test_dataset.classes[dog_label]}')

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Class indices 281-293 are considered correct for label 0 (e.g., "Cat").
# Class indices 151-280 are considered correct for label 1 (e.g., "Dog").

def visual_accuracy(neuralnet, dataloader):
    
    true_prediction = 0
    total_prediction = 0

    device = next(neuralnet.parameters()).device
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = neuralnet(images)
            _, predicted = torch.max(outputs, 1)
            total_prediction += labels.size(0)

            for i in range(images.size(0)):
                true_label = labels[i].item()
                pred_class = predicted[i].item()

                if (281 <= pred_class <= 293 and true_label == 0) or (151 <= pred_class <= 280 and true_label == 1):
                    true_prediction += 1

                img = images[i].cpu().numpy().transpose(1, 2, 0)
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                prob = torch.nn.functional.softmax(outputs, dim=1)[i].cpu().numpy()

                plt.figure(figsize=(5, 5))
                plt.imshow(img)
                plt.title(
                    f"Pred: {pred_class}, True: {'Cat' if(true_label == 0) else 'Dog'}\n"
                    f"Prob: {prob[pred_class]:.2f}\n"
                    f"Prediction: {'Correct' if (281 <= pred_class <= 293 and true_label == 0) or (151 <= pred_class <= 280 and true_label == 1) else 'Incorrect'}"
                )
                plt.axis('off')
                plt.show()

    accuracy = true_prediction / total_prediction
    return accuracy

accuracy = visual_accuracy(neuralnet, test_dataloader)
print(f"Test Accuracy: {accuracy * 100:.2f}%")